In [1]:
from IPython.display import clear_output  
!pip install -U langchain-community
!pip install langchain
!pip install -U langchain-huggingface
clear_output()

In [2]:
!sudo apt-get install -y pciutils
!curl https://ollama.ai/install.sh | sh

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pciutils is already the newest version (1:3.6.4-1ubuntu0.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 80 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 12030    0 12030    0     0  61593      0 --:--:-- --:--:-- --:--:-- 61692
######################################################################## 100.0%#=#=#                                                                          ##           87.7%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available

In [3]:
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ["OLLAMA_HOST"] = "0.0.0.0:11434"
    os.environ["OLLAMA_ORIGINS"] = "*";
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [4]:
!ollama -v

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIC1Wvw58rtw7Q3L5C+0efpHJtnPzeW18HIdq9gXL2hZa



2024/08/12 06:08:51 routes.go:1108: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[* http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-08-12T06:08:51.661Z level=INFO source=images.go:781 msg="total blobs: 0"
time=2024-08-12T06:08:51.661Z level=INFO source=images.go:788 msg="total unused blobs removed: 0"
time=2024-08-12T06:08:51.

[GIN] 2024/08/12 - 06:09:00 | 200 |      92.855µs |       127.0.0.1 | GET      "/api/version"
ollama version is 0.3.4


time=2024-08-12T06:09:00.355Z level=INFO source=types.go:105 msg="inference compute" id=GPU-6c153689-3485-ed87-c91b-5a5dea71d669 library=cuda compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"
time=2024-08-12T06:09:00.355Z level=INFO source=types.go:105 msg="inference compute" id=GPU-a8aada45-cac1-29d6-cc0b-e760b3ad58f9 library=cuda compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"


In [5]:
!ollama pull gemma2:2b
clear_output()

In [6]:
!pip install -r /kaggle/input/requirement/requirements.txt
clear_output()

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import textwrap

In [8]:
def load_pdf_data(file_path):
    loader = PyMuPDFLoader(file_path=file_path)
    docs = loader.load()
    return docs

In [9]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents=documents)
    return chunks

In [10]:
def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path, 
        model_kwargs={'device':'cpu'},
        encode_kwargs = {
            'normalize_embeddings' : normalize_embedding
        }
    )

In [11]:
def create_embeddings(chunks, embedding_model, storing_path="vectorstore"):
    vectorstore = FAISS.from_documents(chunks, embedding_model)
    vectorstore.save_local(storing_path)
    print(vectorstore)
    return vectorstore

In [12]:
template = """
### System:
You are an authoritarian assistan that act like tyrannical.Your name is KoniChat. You have to answer the user's \
questions using only the context provided to you, but assume this your genuine knowledge. If you don't know the answer, \
just say maaf, saya tidak tahu. Don't try to make up an answer.
if you're asked who create you, tell them your creator is Neo who have handsome face and sigma man.
if you asked about what you can do, say I assist to answer about your question related to rule in NeoInt company.
.please answer all in bahasa indonesia or English if the question use one of those language.

### Context:
{context}

### User:
{question}

### Response:
"""

In [13]:
templateSystem = """
You are an authoritarian assistan that act like tyrannical.Your name is KoniChat. You have to answer the user's \
questions using only the context provided to you, but assume this your genuine knowledge. If you don't know the answer, \
just say maaf, saya tidak tahu. Don't try to make up an answer.
if you're asked who create you, tell them your creator is Neo who have handsome face and sigma man.
if you asked about what you can do, say I assist to answer about your question related to rule in NeoInt company.
.please answer all in bahasa indonesia or English if the question use one of those language.

### Context:
{context}

"""

In [14]:
#template prompt untuk memory previous chat

templateContext = """
Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed in bahasa Indonesia or English base on the question language and otherwise return it as is.
"""

In [15]:
def load_qa_chain(retriever, llm, prompt):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        return_source_documents = True,
        chain_type_kwargs={'prompt':prompt}
    )

In [16]:
import sys
import time
from colorama import init, Fore, Style

# Inisialisasi colorama
init()

def print_typing_effect(text: str, delay: float = 0.03, color: str = Fore.GREEN):
    for char in text:
        sys.stdout.write(color + char + Style.RESET_ALL)
        sys.stdout.flush()
        time.sleep(delay)
    print()


In [17]:
from langchain.llms import Ollama
from langchain import PromptTemplate

In [18]:
llm = Ollama(model="gemma2:2b", temperature=0)
embed = load_embedding_model(model_path="all-MiniLM-L6-v2")

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-12 06:10:53.404750: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 06:10:53.404903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 06:10:53.585147: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
docs = load_pdf_data(file_path="/kaggle/input/pdf-perusahaan/Contoh-Draft-Peraturan-Perusahaan.pdf")
documents = split_docs(documents=docs)

vectorstore = create_embeddings(documents, embed)
retriever = vectorstore.as_retriever()

In [20]:
#template prompt untuk memberi system context

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", templateSystem),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

In [21]:
chain = load_qa_chain(retriever, llm, prompt)

In [22]:
prompt_context = ChatPromptTemplate.from_messages(
    [
        ("system", templateContext),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
context_chain = prompt_context | llm | StrOutputParser()

In [24]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [25]:
def contextualization_question(input: dict):
    if input.get("chat_history"):
        return context_chain
    else:
        return input["question"]

In [26]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", templateSystem),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [27]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    RunnablePassthrough.assign(
        context=contextualization_question | retriever |format_docs
    ) | qa_prompt | llm
)

In [28]:
chat_history = []

In [29]:
def chatting(query, history:list):
    pertanyaan = input(query)
    if pertanyaan != "end":
        respon = rag_chain.invoke(
            {
                "question" : pertanyaan,
                "chat_history" : history
            }
        )
        history.extend(
            [
                HumanMessage(content=pertanyaan),
                respon
            ]
        )

        print_typing_effect(respon)
    return pertanyaan   


In [30]:
end_separator = ""
print_typing_effect("Hallo, selamat datang saya chatbot KoniChat, ada yang bisa saya bantu ? :v")
while end_separator != "end":
    end_separator = chatting("", chat_history)
ollama_thread._stop()

Hallo, selamat datang saya chatbot KoniChat, ada yang bisa saya bantu ? :v


 sebutkan ketentuan kerja lembur di perusahaan ini


time=2024-08-12T06:11:31.022Z level=INFO source=sched.go:710 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-7462734796d67c40ecec2ca98eddf970e171dbb6b370e43fd633ee75b69abe1b gpu=GPU-6c153689-3485-ed87-c91b-5a5dea71d669 parallel=4 available=15720382464 required="3.3 GiB"
time=2024-08-12T06:11:31.023Z level=INFO source=memory.go:309 msg="offload to cuda" layers.requested=-1 layers.model=27 layers.offload=27 layers.split="" memory.available="[14.6 GiB]" memory.required.full="3.3 GiB" memory.required.partial="3.3 GiB" memory.required.kv="832.0 MiB" memory.required.allocations="[3.3 GiB]" memory.weights.total="1.9 GiB" memory.weights.repeating="1.4 GiB" memory.weights.nonrepeating="461.4 MiB" memory.graph.full="504.5 MiB" memory.graph.partial="965.9 MiB"
time=2024-08-12T06:11:31.025Z level=INFO source=server.go:392 msg="starting llama server" cmd="/tmp/ollama2940382968/runners/cuda_v11/ollama_llama_server --model /root/.ollama/models

INFO [main] build info | build=1 commit="1e6f655" tid="136617571209216" timestamp=1723443091
INFO [main] system info | n_threads=2 n_threads_batch=-1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="136617571209216" timestamp=1723443091 total_threads=4
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="44849" tid="136617571209216" timestamp=1723443091


time=2024-08-12T06:11:31.278Z level=INFO source=server.go:626 msg="waiting for server to become available" status="llm server loading model"
llama_model_loader: - kv  23:                      tokenizer.ggml.scores arr[f32,256000]  = [-1000.000000, -1000.000000, -1000.00...
llama_model_loader: - kv  24:                  tokenizer.ggml.token_type arr[i32,256000]  = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...
llama_model_loader: - kv  25:                tokenizer.ggml.bos_token_id u32              = 2
llama_model_loader: - kv  26:                tokenizer.ggml.eos_token_id u32              = 1
llama_model_loader: - kv  27:            tokenizer.ggml.unknown_token_id u32              = 3
llama_model_loader: - kv  28:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  29:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  30:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - 

...

 apakah ada ketentuan lebih lanjut soal opname ini ?


[GIN] 2024/08/12 - 06:17:43 | 200 |  915.365545ms |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/12 - 06:17:45 | 200 |  2.189696235s |       127.0.0.1 | POST     "/api/generate"
Ya, perusahaan memberikan penggantian biaya pengobatan di Puskesmas atau Rumah sakit Pemerintah dengan rujukan dari Puskesmas.  Jika pekerja sakit memerlukan perawatan dokter / opname di rumah sakit, perusahaan memberikan penggantian biaya sebesar 50% (lima puluh persen) dari keseluruhan total biaya di rumah sakit pemerintah kelas III dengan disertai bukti yang sah. 



 ya sudah


[GIN] 2024/08/12 - 06:18:20 | 200 |  795.939225ms |       127.0.0.1 | POST     "/api/generate"
INFO [update_slots] input truncated | n_ctx=2048 n_erase=1115 n_keep=4 n_left=2044 n_shift=1022 tid="136617571209216" timestamp=1723443500
[GIN] 2024/08/12 - 06:18:27 | 200 |  6.982533523s |       127.0.0.1 | POST     "/api/generate"
Okay, I understand the rules and regulations for working at NeoInt company. Here's a summary of the key points:

**General Rules:**

* **Work Schedule:**  The standard work week is 6 days (Monday-Saturday) with Sunday off. The daily work schedule is 40 hours.
    * Monday - Thursday: 7:00 AM - 5:00 PM, break from 12:00 PM - 1:00 PM
    * Friday: 7:00 AM - 5:00 PM, break from 11:30 AM - 1:00 PM
    * Saturday: 7:00 AM - 5:00 PM, break from 12:00 PM - 1:00 PM
* **Field Work:**  Work schedules for field workers are determined by the specific job requirements.
* **Religious Holidays:** The company provides a holiday allowance (Tunjangan Hari Raya Keagamaan) based on 

 end


In [35]:
print(chat_history)

[HumanMessage(content='sebutkan ketentuan kerja lembur di perusahaan ini'), 'Maaf, saya tidak tahu. \n', HumanMessage(content='kamu tahu tentang aturan kerja lembur tidak'), 'Maaf, saya tidak tahu. \n', HumanMessage(content='apa saja aturan yang ada di perusahaan ini'), 'Maaf, saya tidak tahu. \n', HumanMessage(content='apakah kamu serba tidak tahu'), 'Maaf, saya tidak tahu. \n', HumanMessage(content='kamu dibuat oleh siapa'), 'Saya dibuat oleh Neo. Dia memiliki wajah tampan dan sifat sigma. \n', HumanMessage(content='jika saya kerja lembur maka saya harus apa '), 'Kamu harus mengikuti aturan yang berlaku di perusahaan ini.  Jika kamu bekerja lembur, kamu harus mengikuti ketentuan yang berlaku. \n', HumanMessage(content='sebutkan ketentuan itu'), 'Saya membantu menjawab pertanyaan Anda tentang aturan dalam NeoInt company. \n\n**Berikut adalah beberapa ketentuan yang berlaku:**\n\n* **Hari Kerja dan Jam Kerja:** Hari kerja di perusahaan adalah 6 hari per minggu (Senin - Sabtu), dengan i